In [3]:
import numpy as np
from functools import reduce
from scipy.linalg import eigh
import matplotlib.pyplot as plt
from itertools import product

# Define Pauli matrices
def pauli_x():
    return np.array([[0, 1], [1, 0]])

def pauli_z():
    return np.array([[1, 0], [0, -1]])

def pxp(N, pbc=False):
    """Constructs the PXP Hamiltonian."""
    I = np.eye(2)
    X = pauli_x()
    Z = pauli_z()
    P = 0.5 * (I - Z)
    H = np.zeros((2**N, 2**N), dtype=np.float64)
    for i in range(1, N-1):
        term = 1
        for j in range(N):
            if j == i:
                term = np.kron(term, X)
            elif j == i - 1 or j == i + 1:
                term = np.kron(term, P)
            else:
                term = np.kron(term, I)
        H += term
    return H

# Construct PXP Hamiltonian in the constrained Rydberg blockade subspace
def construct_pxp_hamiltonian(n_spins):
    """Constructs the PXP Hamiltonian matrix in the constrained subspace."""
    full_H = pxp(n_spins, pbc=False)
    full_basis = list(product([0, 1], repeat=n_spins))
    allowed_basis = [state for state in full_basis if all(state[i] + state[i + 1] <= 1 for i in range(n_spins - 1))]
    basis_size = len(allowed_basis)
    state_to_index = {state: idx for idx, state in enumerate(allowed_basis)}
    H = np.zeros((basis_size, basis_size), dtype=float)
    for i, state_i in enumerate(allowed_basis):
        for j, state_j in enumerate(allowed_basis):
            idx_i = int("".join(map(str, state_i)), 2)
            idx_j = int("".join(map(str, state_j)), 2)
            H[i, j] = full_H[idx_i, idx_j]
    return H, allowed_basis

# Custom function to compute the partial trace
def partial_trace(rho, keep, dims):
    """Compute the partial trace of a density matrix."""
    keep_dims = np.prod([dims[i] for i in keep])
    trace_dims = np.prod([dims[i] for i in range(len(dims)) if i not in keep])
    rho = rho.reshape([keep_dims, trace_dims, keep_dims, trace_dims])
    return np.trace(rho, axis1=1, axis2=3).reshape([keep_dims, keep_dims])

# Define number of spins
N_full = 8
N_reduced = 3

# Compute PXP Hamiltonian in constrained subspace
H_PXP_full = pxp(N_full, pbc=True)

# Compute ground state
eigvals, eigvecs = eigh(H_PXP_full)
psi_0 = eigvecs[:, 0]  # Ground state

# Construct full density matrix
rho_0 = np.outer(psi_0, psi_0.conj())

# Compute partial trace to reduce to 3 spins
dims = [2] * N_full
rho_red = partial_trace(rho_0, keep=[0, 1, 2], dims=dims)

# Restrict Casimir operator to the 3-spin subspace
Q_x_red = pxp(N_reduced, pbc=False)
Casimir_red = np.dot(Q_x_red, Q_x_red)

# Check commutation of reduced density matrix with restricted Casimir operator
comm_rho_Casimir = np.dot(rho_red, Casimir_red) - np.dot(Casimir_red, rho_red)

# Print results
print("[Reduced Density Matrix, Restricted Casimir] =\n", comm_rho_Casimir)

[Reduced Density Matrix, Restricted Casimir] =
 [[ 0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
  -0.17840102  0.        ]
 [ 0.          0.          0.          0.          0.          0.17840102
   0.         -0.22813597]
 [ 0.          0.          0.          0.          0.          0.
   0.22813597  0.        ]]
